<a href="https://colab.research.google.com/github/artistho/Experimeting-OpenVINO/blob/master/keras2OpenVINO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Install<br>
Tensorflow 2.0 <br>
Access to gdrive 

In [0]:
# Install TF 2.0 
!pip install -q tensorflow==2.0.0-alpha0

In [0]:
# Install access to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

In [23]:
# pydot is an interface to GraphViz(used to render graphs as PDF, PNG, SVG, etc.)
!pip install -q pydot
!apt-get install graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


#####Import the required libraries

In [27]:
# To write a Python 2/3 compatible codebase
from __future__ import absolute_import, division, print_function, unicode_literals

# TF keras lib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# To interact with file system
from pathlib import Path

print("tf version is {} and ".format(tf.__version__), "keras version is {}".format(keras.__version__))

tf version is 2.0.0-alpha0 and  keras version is 2.2.4-tf


#####Create dir in gdrive

In [0]:
# Create respective directories in gdrive
root_dir = '/content/gdrive/My Drive/'
base_dir = root_dir + '/TF/'
path_data = Path(base_dir + 'data')
dest = path_data
dest.mkdir(parents = True, exist_ok = True)


#####Create the model

In [0]:
# For easy reset of notebook state
tf.keras.backend.clear_session()

In [0]:
# Create an input node.
# Shape is 784(28 * 28), shape of mnist img.
inputs = keras.Input(shape = (784))

In [29]:
# Create 2 fully connected layers with 64 activation neurons.  Activation function is relu.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)

# Create 1(final) fully connected layer with 10 activation neurons.  Activation function is softmax.
outputs = layers.Dense(10, activation='softmax')(x)

# Final model creation
model = keras.Model(inputs=inputs, outputs=outputs, name = "minist_model")

# Print the model
model.summary()

Model: "minist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


#####Load MNIST data

In [0]:
# Load data from data available in keras datasets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Divide by 255, this is the maximum value of a byte (the input feature's type before the conversion to float32), 
# so this will ensure that the input features are scaled between 0.0 and 1.0. 
# At one point the training data will have to be converted to 32 bit floats, we converted immediately first(the
# data fits into RAM).  

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255


#####Training and evaluation loop

In [0]:
# Configure the learning process by adding loss, optimizer and metrics to the arguments.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

In [34]:
# Fitting or training.
history = model.fit(x_train, y_train, # data
                    batch_size=64, # size of each batch
                    epochs=6, # number of epochs
                    validation_split=0.2) # create validation data with 20% of x_train datasets

Train on 48000 samples, validate on 12000 samples
Epoch 1/6
48000/48000 [==============================] - 2s 50us/sample - loss: 0.3501 - accuracy: 0.9025 - val_loss: 0.1870 - val_accuracy: 0.9481
Epoch 2/6
48000/48000 [==============================] - 2s 48us/sample - loss: 0.1676 - accuracy: 0.9502 - val_loss: 0.1501 - val_accuracy: 0.9557
Epoch 3/6
48000/48000 [==============================] - 2s 49us/sample - loss: 0.1231 - accuracy: 0.9638 - val_loss: 0.1277 - val_accuracy: 0.9632
Epoch 4/6
48000/48000 [==============================] - 2s 48us/sample - loss: 0.0998 - accuracy: 0.9704 - val_loss: 0.1127 - val_accuracy: 0.9678
Epoch 5/6
48000/48000 [==============================] - 2s 48us/sample - loss: 0.0829 - accuracy: 0.9747 - val_loss: 0.1084 - val_accuracy: 0.9689
Epoch 6/6
48000/48000 [==============================] - 2s 48us/sample - loss: 0.0709 - accuracy: 0.9784 - val_loss: 0.1039 - val_accuracy: 0.9689


In [56]:
# .evaluate() computes the loss based on the input you pass it, along with any other metrics that you 
#   requested in the metrics param when you compiled your model.
test_scores = model.evaluate(x_test, y_test, verbose=0)

# Printing the test results.
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.09543134405561723
Test accuracy: 0.9714


In [0]:
# .predict() generates output predictions based on the input you pass it.
predictions = model.predict(x_test)

#####Save the model in .h5

In [0]:
# Save the model in .h5 format
model.save(path_data / "keras2OpenVINO.h5")

In [0]:
# Recreate the exact same model 
check_model = keras.models.load_model(path_data / "keras2OpenVINO.h5")

In [0]:
import numpy as np
# Check that the state is preserved
check_predictions = check_model.predict(x_test)
np.testing.assert_allclose(predictions, check_predictions, atol=1e-6)

#####Save the model in .pb

In [62]:
# Export the model to a saved_model(serialized)
keras.experimental.export_saved_model(model, '/content/gdrive/My Drive/TF/keras2OpenVINO')


W0415 08:23:21.092509 140324652828544 tf_logging.py:161] Export includes no default signature!
W0415 08:23:21.710744 140324652828544 tf_logging.py:161] Export includes no default signature!


In [0]:
# Recreate the model
new_model = keras.experimental.load_from_saved_model('/content/gdrive/My Drive/TF/keras2OpenVINO')

# Check that the state is preserved
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, atol=1e-6)